In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Dropout,BatchNormalization,Layer,GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline
import time

In [2]:
#Colocar coisa da microsoft
import os
import azureml
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import Workspace, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ModuleNotFoundError: No module named 'ruamel'

In [ ]:
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        positiveDist = K.sum(K.square(anchor-positive), axis=-1)
        negativeDist = K.sum(K.square(anchor-negative), axis=-1)
        return K.sum(K.maximum(positiveDist - negativeDist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

In [ ]:
def build_network(input_shape, embeddingsize):
    # Adding layers in the Convolutional Neural Network
    network = Sequential()
    network.add(Conv2D(128, (7,7), activation='relu',input_shape=input_shape,kernel_initializer='he_uniform',kernel_regularizer=l2(2e-4)))
    network.add(MaxPooling2D())
    network.add(Conv2D(128, (3,3), activation='relu', kernel_initializer='he_uniform',kernel_regularizer=l2(2e-4)))
    network.add(MaxPooling2D())
    network.add(Conv2D(256, (3,3), activation='relu', kernel_initializer='he_uniform',kernel_regularizer=l2(2e-4)))
    network.add(GlobalAveragePooling2D())
    network.add(Dense(4096, activation='relu',kernel_regularizer=l2(1e-3),kernel_initializer='he_uniform'))
    
    # Layer to convert all images to the fixed embedding size, to compute distance
    network.add(Dense(embeddingsize, activation=None,kernel_regularizer=l2(1e-3),kernel_initializer='he_uniform'))
    
    # Force the encoding to live on the d-dimentional hypershpere
    network.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))
    
    return network

In [ ]:
def build_model(inputShape, network, margin=0.2):
    # Building the model
    
    # Define the tensors for the three input images in the triplet loss layer
    anchorInput = Input(inputShape, name="anchorInput")
    positiveInput = Input(inputShape, name="positiveInput")
    negativeInput = Input(inputShape, name="negativeInput") 
    
    # Generate the encodings (feature vectors) for the three images
    encodedA = network(anchorInput)
    encodedP = network(positiveInput)
    encodedN = network(negativeInput)
    
    #TripletLoss Layer, three feature vectores (images) as input
    # margin, float value to ensure distance wont be never 0
    lossLayer = TripletLossLayer(alpha=margin,name='tripletLossLayer')([encodedA,encodedP,encodedN])
    
    # Connect the inputs with the outputs, creating model
    model = Model(inputs=[anchorInput,positiveInput,negativeInput],outputs=lossLayer)
    
    # Defining learning rate
    optimizer = Adam(lr = 0.0006)
    
    #compiling model, loss none cause last layer is already the loss
    model.compile(loss=None,optimizer=optimizer)
    
    return model

In [ ]:
def computeDist(a,b):
    return np.sum(np.square(a-b))

def computeProbs(network,X,Y):
    # Compute probabilities of similiraty for all images in X
    
    numberOfImages = len(X)
    
    # Get number of comparisons and intiliaze empty vectos for probability and real target
    nbevaluation = int(m*(m-1)/2)
    probs = np.zeros((nbevaluation))
    y = np.zeros((nbevaluation))
    
    # For all images received in X, get their embeddings ( Feature vector, representation of image)
    embeddings = network.predict(X)
    
    # get distance for each image from all other images
    k = 0
    for i in range(numberOfImages):
            #Against all other images
            for j in range(i+1,numberOfImages):
                #compute the probability of being the right decision :  1 for right class, 0 for all other classes
                probs[k] = computeDist(embeddings[i,:],embeddings[j,:])
                
                # compare to see the real result from Y_test 
                if (Y[i]==Y[j]):
                    y[k] = 0
                else:
                    y[k] = 1
                k += 1
                
    return probs,y


def computeAccuracy(probs,y):
    # Based on probs generate from computeProbs and y_test array, create score with percentual of cases correct
    
    correctPrediction = 0
    totalPredictions = len(probs)
    probThreshholded = []
    
    # Threshold value to decide wether images are equal or different
    threshhold = 0.5
    
    # Creating vector with predicitions 0 or 1, after passing through threshold
    # Check if np.clip does the same
    for prob in probs:
        if(prob > 0.5):
            probThreshholded.append(1)
        else:
            probThreshholded.append(0)
            
    for index,prob in enumerate(probThreshholded):
        # If prediction equal to true target, add 1 to numer of corrects predictions
        if(prob == y[index]): 
            correctPrediction += 1
            
    return correctPrediction/totalPredictions # Percentage of correct cases

In [ ]:
%run DatasetPrepare.ipynb

In [ ]:
imgsPath = os.getcwd()+'/modelHB_imgs/train/'
csvPath = 'train_HB.csv'
createLabelsTargets = CreateLabelsTargets(csvPath)
createLabelsTargets.load_csv_and_treat()
(X_train, X_test_original, y_train, y_test_original) = createLabelsTargets.create_dataset_with_image(imgsPath)

In [ ]:
inputShape = (200,200,1)
embeddingsize = 64
network = build_network(inputShape, embeddingsize)
model = build_model(inputShape, network, margin=0.2)
dataLoader =DataLoader('',X_train,y_train)
model.summary()

In [ ]:
evaluateEvery = 100 # interval for evaluating the model with test images
batchSize = 32
epochs = 1000 # Number of training iterations
nVal = 250 # How many triples to validate on

In [ ]:
# Rehasping X_test to work on compute probs funtction
X_test = []
y_test = []
for index,values in enumerate(X_test_original):
    for value in values:
        X_test.append(value)
        y_test.append(y_test_original[index])
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
print("Training...")
t_start = time.time()

for i in range(1, epochs):
    triplets = dataLoader.getBatchRandom(batchSize,X_train)
    loss = model.train_on_batch(triplets, None)
    if i % evaluateEvery == 0:
        print("Evaluating")
        print("\n ------------- \n")
        
        # Sorting nVal images to validate on
        randomIndex = rng.randint(len(y_test))
        if(randomIndex < nVal):
            randomIndex += (nVal - randomIndex)
        
        #compute probabilities
        probs,yprob = computeBrobs(network,X_test[(randomIndex-nVal):randomIndex],y_test[(randomIndex-nVal):randomIndex])
        # compute accuracy
        acc = computeAccuracy(probs,yprob)
        print("[{3}] Time for {0} iterations: {1:.1f} mins, Train Loss: {2}".format(i, (time.time()-t_start)/60.0,loss,i))
        print("[{}] Validating in test set[{},{}] Test_set Accuracy: {}".format(i,randomIndex-nVal, randomIndex, acc))

In [ ]:
probs,yprob = compute_probs(network,X_test,y_test)
acc = computeAccuracy(probs,yprob)

In [ ]:
acc

In [ ]:
probs,yprob = compute_probs(network,X_test[0:10],y_test[0:10])
print("probs")
print(probs)
print("y")
print(yprob)
acc = computeAccuracy(probs,yprob)

In [ ]:
compute_dist(probas[2256],probas[2257])

In [ ]:
array_to_img(X_test[2256])

In [ ]:
array_to_img(X_test[2257])

In [ ]:
y_test[2255:2264]

In [ ]:
network.save_weights('HBWhale_Model_Weights.h5')

In [ ]:
model.load_weights('HBWhale_Model_Weights.h5')